<a href="https://colab.research.google.com/github/inventor-2-developer/homes_sales/blob/main/Home_Sales_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [47]:
# Import packages
from pyspark.sql import SparkSession
from pyspark import SparkFiles
import time
import re
from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.sql.functions import avg
from pyspark.sql.functions import round
from pyspark.sql.functions import year,col
from pyspark.sql.types import IntegerType,BooleanType,DateType

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [48]:
# Read in the dataset from the AWS S3 bucket.
url = "https://2u-data-curriculum-team.s3.amazonaws.com/nflx-data-science-adv/week-5/Assignment/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df=spark.read.csv(SparkFiles.get("home_sales_revised.csv"),sep=",",header=True)
df.show()

+---+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|_c0|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+---+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|  0|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|  1|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|  2|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|  3|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|  4|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|  5|5aa00529-0533-46b...|2019-01-30|      2017|

In [49]:
df.select(year(df['date'])).show()

+----------+
|year(date)|
+----------+
|      2022|
|      2021|
|      2019|
|      2019|
|      2022|
|      2019|
|      2020|
|      2019|
|      2020|
|      2021|
|      2019|
|      2021|
|      2020|
|      2021|
|      2019|
|      2021|
|      2021|
|      2019|
|      2019|
|      2019|
+----------+
only showing top 20 rows



In [50]:
df=df.withColumn("year",year(df['date']))
df.show()

+---+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|_c0|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+---+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|  0|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|
|  1|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|2021|
|  2|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|2019|
|  3|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|2019|
|  4|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|2022|
|  5|5aa

In [61]:
#df2 = df.withColumn("date_built"col("date_built").cast(IntegerType()))",
   # "cast(isGraduated as string) isGraduated",
   # "cast(jobStartDate as string) jobStartDate")


df2 = df.withColumn("date_built",df.date_built.cast(IntegerType()))\
    .withColumn("price",df.price.cast(IntegerType()))\
    .withColumn("bedrooms",df.bedrooms.cast(IntegerType()))\
    .withColumn("bathrooms",df.bathrooms.cast(IntegerType()))\
    .withColumn("sqft_living",df.sqft_living.cast(IntegerType()))\
    .withColumn("sqft_lot",df.sqft_lot.cast(IntegerType())) \
    .withColumn("floors",df.floors.cast(IntegerType()))\
    .withColumn("waterfront",df.waterfront.cast(IntegerType()))\
    .withColumn("view",df.view.cast(IntegerType()))\







In [62]:
df2.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- year: integer (nullable = true)



In [155]:
# 1. Create a temporary view of the DataFrame.
df2.createOrReplaceTempView('home_sales')

In [214]:
# 2. What is the average price for a four bedroom house sold in 2019 rounded to two decimal places?
#sql="""SELECT price FROM home_sales   """
#spark.sql(sql).show()

spark.sql("select round(AVG(price),2) as AVERAGE_PRICE from home_sales where rlike(date,'(2019)')and bedrooms==4").show()




+-------------+
|AVERAGE_PRICE|
+-------------+
|     300263.7|
+-------------+



In [213]:
# did this question two ways and got the same answer
spark.sql("select round(AVG(price),2) as AVERAGE_PRICE from home_sales where year ==2019 and bedrooms==4").show()

+-------------+
|AVERAGE_PRICE|
+-------------+
|     300263.7|
+-------------+



In [212]:
# 3. What is the average price of a home for each year that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

sql=("""SELECT year,round(AVG(price),2) as AVERAGE_PRICE FROM home_sales WHERE((bathrooms==3) AND (bedrooms==3)) GROUP BY year ORDER BY year desc """)
spark.sql(sql).show()


+----+-------------+
|year|AVERAGE_PRICE|
+----+-------------+
|2022|    292725.69|
|2021|    294211.46|
|2020|    294204.16|
|2019|    287287.82|
+----+-------------+



In [211]:
# 4. What is the average price of a home for each year that have 3 bedrooms, 3 bathrooms, and are less than 2,000 square feet rounded to two decimal places?
sql=("""SELECT year,round(AVG(price),2) as AVERAGE_PRICE FROM home_sales WHERE((bathrooms==3) AND (bedrooms==3) AND (sqft_living<2000)) GROUP BY year ORDER BY year desc """)

spark.sql(sql).show()


+----+-------------+
|year|AVERAGE_PRICE|
+----+-------------+
|2022|    283031.02|
|2021|     282452.7|
|2020|    291973.53|
|2019|    280167.38|
+----+-------------+



In [215]:
# 5. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are less than $350,000?
# Although this is a small dataset, determine the run time for this query.
start_time = time.time()
sql=("""SELECT view,round(AVG(price),2) as AVERAGE_PRICE FROM home_sales WHERE((price>350000)) GROUP BY view ORDER BY view desc """)
spark.sql(sql).show()
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.3129312992095947 seconds ---


In [200]:
# 6. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [202]:
# 7. Check if the table is cached.
spark.catalog.isCached("home_sales")

True

In [216]:
# 8. Using the cached data, run the query that filters out the view ratings with average price of less than $350,000. 
# Determine the runtime and compare it to uncached runtime.

start_time = time.time()

sql=("""SELECT view,round(AVG(price),2) as AVERAGE_PRICE FROM home_sales WHERE((price>350000)) GROUP BY view ORDER BY view desc """)
spark.sql(sql).show()
print("--- %s seconds ---" % (time.time() - start_time))




+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.42626190185546875 seconds ---


In [204]:
# 9. Partition the home sales dataset by the date_built field.
df2.write.partitionBy("date_built").mode("overwrite").parquet("home_sales")

In [205]:
# 10.Read the formatted parquet data.
p_df_p=spark.read.parquet('home_sales')

In [206]:
# 11. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_home_sales_p')

In [207]:
# 12. Run the query that filters out the view ratings with average price of less than $350,000 with the parquet DataFrame. 
# Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 
start_time = time.time()

sql=("""SELECT view,AVG(price) as AVERAGE_PRICE FROM p_home_sales_p WHERE((price>350000)) GROUP BY view ORDER BY view desc """)
spark.sql(sql).show()
print("--- %s seconds ---" % (time.time() - start_time))




+----+------------------+
|view|     AVERAGE_PRICE|
+----+------------------+
| 100|         1026669.5|
|  99|1061201.4193548388|
|  98|1053739.3333333333|
|  97| 1129040.148148148|
|  96|1017815.9166666666|
|  95|         1054325.6|
|  94|         1033536.2|
|  93|1026006.0606060605|
|  92| 970402.5476190476|
|  91| 1137372.731707317|
|  90|1062654.1621621621|
|  89|        1107839.15|
|  88|1031719.3548387097|
|  87|         1072285.2|
|  86|        1070444.25|
|  85|1056336.7435897435|
|  84|1117233.1290322582|
|  83|       1033965.925|
|  82|         1063498.0|
|  81|1053472.7878787878|
+----+------------------+
only showing top 20 rows

--- 0.8567702770233154 seconds ---


In [208]:
# 13. Uncache the home_sales temporary table.

spark.sql("uncache table p_home_sales_p")

DataFrame[]

In [210]:
# 14. Check if the home_sales is no longer cached
spark.catalog.isCached("p_home_sales_p")

False